# Power consumption prediction from smart meters data using ARIMA

## Loading and preparing data set.
Tha data has been already passed from the original JSON file where it is stored by the concentrator to a CSV file which can be read directly by Pandas.
We need to import all the required Pandas,SciKitLearn and Numpy dependencies:
 

In [1]:
import pandas as pd
from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error


Then we get the generic dependencies used for the script

In [2]:
import sys, getopt, re
from datetime import datetime

Then we read th CSV file, set the Fh column as datetime and then sort the rows by datetime (we do not know if the order is right before processig it):

In [11]:
samples02_df = pd.read_csv('/home/anton/src/smart_meter_data_analysis/arima_meter_data_consumption_prediction/meter_data_ZIV0035301588.csv')
samples02_df['Fh'] = pd.to_datetime(samples02_df['Fh'])
samples02_df= samples02_df.sort_values(by=['Fh'])

Then we set Fh column as index:

In [12]:
samples02_df = samples02_df.set_index('Fh')
print(samples02_df.head())

AI   R1  R4  Bc
Fh                                   
2016-12-14 23:00:00  500  307   0   0
2016-12-15 00:00:00  413  274   0   0
2016-12-15 01:00:00  280  155   0   0
2016-12-15 02:00:00  237  336   0   0
2016-12-15 03:00:00  245  352   0   0


We assume we do not have missing values so we do not need to fill it.

We are going to try to estimate the power consumption one week ahead. *This requires that a predictive model forecast the total active power for each day over the next seven days.*

*Technically, this framing of the problem is referred to as a multi-step time series forecasting problem, given the multiple forecast steps. A model that makes use of multiple input variables may be referred to as a multivariate multi-step time series forecasting model.* The sample data gives the consumption pero hour so we do not need to do a re-sampling.

We need to define a metric to evaluate the performance of the model. In our case we use the RMSE for the deinfe period. In our case it will be a week but there is not any limitation for the period length. 

In [ ]:

# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
	scores = list()
	# calculate an RMSE score for each day
	for i in range(actual.shape[1]):
		# calculate mse
		mse = mean_squared_error(actual[:, i], predicted[:, i])
		# calculate rmse
		rmse = sqrt(mse)
		# store
		scores.append(rmse)
	# calculate overall RMSE
	s = 0
	for row in range(actual.shape[0]):
		for col in range(actual.shape[1]):
			s += (actual[row, col] - predicted[row, col])**2
	score = sqrt(s / (actual.shape[0] * actual.shape[1]))
	return score, scores

This funtion display the score of the model so that it can be easily compared with others

In [ ]:
# summarize scores
def summarize_scores(name, score, scores):
	s_scores = ', '.join(['%.1f' % s for s in scores])
	print('%s: [%.3f] %s' % (name, score, s_scores))

This function divides the datasate into training and test. ANTON NEEDS TO ADAPT TO YOUR DATA AND ADD MORE PARAMETERS!!!!

In [ ]:
# split a univariate dataset into train/test sets
def split_dataset(data):
	# split into standard weeks
	train, test = data[1:-328], data[-328:-6]
	# restructure into windows of weekly data
	train = array(split(train, len(train)/7))
	test = array(split(test, len(test)/7))
	return train, test